In [1]:
from pyps import Basis, Hamiltonian, eigenenergies
from pyps.constants import e, a0, En_h, mu_B

In [2]:
H = Hamiltonian(Basis(n_values=[1]), sparse_format="csc")

## Field-free Hamiltonian

In [3]:
H.e0()

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Column format>

In [4]:
H.e0().todense()

matrix([[-0.25001747,  0.        ,  0.        ,  0.        ],
        [ 0.        , -0.24998641,  0.        ,  0.        ],
        [ 0.        ,  0.        , -0.24998641,  0.        ],
        [ 0.        ,  0.        ,  0.        , -0.24998641]])

In [5]:
# diagonal elements
H.e0().diagonal()

array([-0.25001747, -0.24998641, -0.24998641, -0.24998641])

In [6]:
# i.e.
H.eigenenergies(units='atomic_units')

array([-0.25001747, -0.24998641, -0.24998641, -0.24998641])

In [7]:
# alt.
eigenenergies(H.e0())

array([-0.25001747, -0.24998641, -0.24998641, -0.24998641])

## Stark matrix

In [8]:
H.stark(Fz=500.1)

calculate Stark terms: 100%|████████████████████| 4/4 [00:00<00:00, 5152.71it/s]


<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>

## Zeeman matrix

In [9]:
H.zeeman(Bz=150.0)

calculate Zeeman terms: 100%|████████████████████| 4/4 [00:00<00:00, 582.20it/s]


<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Column format>

## Total Hamiltonian 

The underlying Stark and Zeeman terms have been cached for rapid recalculation.

In [10]:
H.total(Fz=10.9, Bz=0.1)

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Column format>

However, the cache can be cleared using the `reset` method.

In [11]:
H.reset()

... the terms are subsequently recalculated when needed.

In [12]:
H.total(Fz=10, Bz=.1, numerov=True)

calculate Zeeman terms: 100%|███████████████████| 4/4 [00:00<00:00, 3869.28it/s]


<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Column format>

Total Hamiltonian matrix converted to an array.

In [13]:
H.total(Bz=.1).toarray()

array([[-0.25001747,  0.        ,  0.2       ,  0.        ],
       [ 0.        , -0.24998641,  0.        ,  0.        ],
       [ 0.2       ,  0.        , -0.24998641,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.24998641]])

Eigenvalues are computed from dense matrices (requires more memory but faster than sparse calc.)

In [14]:
# eigenvalues
electric_field = 582  # V/m
magnetic_field = 0.2  # T
H.eigenenergies(electric_field, magnetic_field, units="eV")

array([-6.80332261, -6.8024767 , -6.8024767 , -6.80247606])

In [15]:
# alt.
Fz = electric_field * e * a0 / En_h
Bz = magnetic_field * mu_B / En_h
eigenenergies(H.e0() + H.stark(Fz) + H.zeeman(Bz), units="eV")

array([-6.80332261, -6.8024767 , -6.8024767 , -6.80247606])

In [16]:
M = H.total(Bz=.1)
M

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Column format>

In [17]:
type(M)

scipy.sparse.csc.csc_matrix

## dense matrices


In [18]:
H.reset()
H.sparse_format = "array"
M = H.total(Fz=10, Bz=0.1, numerov=True)
M

calculate Zeeman terms: 100%|███████████████████| 4/4 [00:00<00:00, 5819.36it/s]


array([[-0.25001747,  0.        ,  0.2       ,  0.        ],
       [ 0.        , -0.24998641,  0.        ,  0.        ],
       [ 0.2       ,  0.        , -0.24998641,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.24998641]])

In [19]:
type(M)

numpy.ndarray

In [20]:
H.e0()

array([[-0.25001747,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.24998641,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.24998641,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.24998641]])